In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines3[extra]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
freeglut3-dev set to manually installed.
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.9 [784 kB]
Fetched 784 kB in 0s (8,337 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.9_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.9) ...
Setting up xvfb (2:1.19.6-1ubuntu4.9) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 172 kB 13.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -e /content/drive/Othercomputers/My\ Laptop/PortfolioAllocationRL/PortfolioAllocation
!

Obtaining file:///content/drive/Othercomputers/My%20Laptop/PortfolioAllocationRL/PortfolioAllocation
  Attempting uninstall: PortfolioAllocation
    Found existing installation: PortfolioAllocation 0.1
    Can't uninstall 'PortfolioAllocation'. No files were found to uninstall.
  Running setup.py develop for PortfolioAllocation


In [ ]:
import gym
import PortfolioAllocationGym
import numpy as np

In [ ]:
from stable_baselines3.common.monitor import Monitor

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [9]:
env_kwargs = {'filename':'/content/drive/Othercomputers/My Laptop/PortfolioAllocationRL/sp500.csv',
    'date_from':'2008-01-01',
    'date_to':'2017-12-31',
#    'date_to':'2009-12-31',
    'investment':1000000,
    'risk_free_rate': 0.5, # approx US Treasury Note return
    'sample_size':500,
    #'report_point':252, # 1 year
    'reward_function':'sharpe'}

train_env = gym.make('PortfolioAllocation-v0', **env_kwargs)


In [10]:
from stable_baselines3.common.env_checker import check_env
check_env(train_env)
train_env, obs = train_env.get_sb_env()

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/env_checker.py:131: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  f"Your observation {key} has an unconventional shape (neither an image, nor a 1D vector). "
/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/env_checker.py:273: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


In [11]:
from stable_baselines3 import A2C
from stable_baselines3.a2c import MlpPolicy
import torch as th

In [12]:
'''
a2c_model = A2C(policy = MlpPolicy,
                use_sde = True,
                env = venv, #train_env,
                tensorboard_log = 'tensorboard',  
                verbose = 0,
                n_steps = 5, 
                ent_coef = 0.05,
                learning_rate =1.0197314719907328e-05
                )
                Trial 20 finished with value: 429.970813 and parameters: {'gamma': 0.995, 'normalize_advantage': True, 'max_grad_norm': 0.8, 'use_rms_prop': False, 'gae_lambda': 0.9, 'n_steps': 10, 'lr_schedule': 'constant', 'learning_rate': 0.8299158561899722, 'ent_coef': 2.1978322752366113e-06, 'vf_coef': 0.49045640410974656, 'log_std_init': -2.594815769134923, 'ortho_init': True, 'net_arch': 'medium', 'sde_net_arch': 'tiny', 'activation_fn': 'elu'}. Best is trial 20 with value: 429.970813.
'''
policy_kwargs = dict(log_std_init = -2.3266038564673126, ortho_init = True, net_arch=[dict(pi=[64, 64], vf=[64, 64])], sde_net_arch =[64, 64], activation_fn =th.nn.LeakyReLU)
 
a2c_model = A2C(policy = MlpPolicy,
                env = train_env,
                 gamma = 0.9,
                 normalize_advantage =True,
                 max_grad_norm= 0.7, 
                 use_rms_prop = True, 
                 gae_lambda = 1.0, 
                 n_steps = 10, 
                 learning_rate=1.0197314719907328e-05, 
                 ent_coef=1.6292796291623184e-05, 
                 vf_coef = 0.39756278734900274,
                 policy_kwargs=policy_kwargs)


In [13]:
from PortfolioAllocationGym.callbacks import TensorBoardCallback as tbc
from datetime import datetime

In [14]:
# Random Agent, before training
mean_reward, std_reward = evaluate_policy(a2c_model, train_env, n_eval_episodes=10)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

day: 2516                 sharpe: 0.382                  psr: 0.070                  cum. rtns: 275.702                 portf val: 3,757,024.02
day: 2516                 sharpe: 0.379                  psr: 0.070                  cum. rtns: 258.732                 portf val: 3,587,323.52
day: 2516                 sharpe: 0.377                  psr: 0.070                  cum. rtns: 258.090                 portf val: 3,580,903.96
day: 2516                 sharpe: 0.377                  psr: 0.070                  cum. rtns: 258.177                 portf val: 3,581,766.38
day: 2516                 sharpe: 0.376                  psr: 0.070                  cum. rtns: 258.193                 portf val: 3,581,933.71
day: 2516                 sharpe: 0.376                  psr: 0.070                  cum. rtns: 258.238                 portf val: 3,582,379.72
day: 2516                 sharpe: 0.376                  psr: 0.070                  cum. rtns: 258.264                 portf val: 3,582

In [18]:
total_timesteps = 100 * (len(train_env.venv.venv.envs[0].data.date.unique())-1)

trained_a2c_model= a2c_model.learn(total_timesteps=total_timesteps,
                                   tb_log_name='A2C'+datetime.now().strftime("%H-%M"))

day: 2516                 sharpe: 0.409                  psr: 0.069                  cum. rtns: 257.695                 portf val: 3,576,945.02
day: 2516                 sharpe: 0.380                  psr: 0.070                  cum. rtns: 286.687                 portf val: 3,866,867.76
day: 2516                 sharpe: 0.332                  psr: 0.068                  cum. rtns: 287.385                 portf val: 3,873,845.52
day: 2516                 sharpe: 0.374                  psr: 0.068                  cum. rtns: 257.719                 portf val: 3,577,188.28
day: 2516                 sharpe: 0.409                  psr: 0.069                  cum. rtns: 283.393                 portf val: 3,833,929.72
day: 2516                 sharpe: 0.394                  psr: 0.069                  cum. rtns: 254.165                 portf val: 3,541,649.82
day: 2516                 sharpe: 0.379                  psr: 0.067                  cum. rtns: 263.712                 portf val: 3,637

In [17]:
mean_reward, std_reward = evaluate_policy(trained_a2c_model, train_env, n_eval_episodes=10)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

day: 2516                 sharpe: 0.461                  psr: 0.070                  cum. rtns: 247.017                 portf val: 3,470,174.02
day: 2516                 sharpe: 0.461                  psr: 0.070                  cum. rtns: 247.009                 portf val: 3,470,093.53
day: 2516                 sharpe: 0.461                  psr: 0.070                  cum. rtns: 247.000                 portf val: 3,470,001.42
day: 2516                 sharpe: 0.461                  psr: 0.070                  cum. rtns: 246.992                 portf val: 3,469,916.02
day: 2516                 sharpe: 0.461                  psr: 0.070                  cum. rtns: 246.986                 portf val: 3,469,855.68
day: 2516                 sharpe: 0.461                  psr: 0.070                  cum. rtns: 246.981                 portf val: 3,469,812.31
day: 2516                 sharpe: 0.461                  psr: 0.070                  cum. rtns: 246.974                 portf val: 3,469

In [ ]:
eval_kwargs = {'filename':'/content/drive/Othercomputers/My Laptop/PortfolioAllocationRL/sp500.csv',
    'date_from':'2018-01-01',
    'date_to':'2020-12-31',
#    'date_to':'2009-12-31',
    'investment':1000000,
    'risk_free_rate': 0.5, # approx US Treasury Note return
    'sample_size':500,
    #'report_point':252, # 1 year
    'reward_function':'sharpe'}

eval_env =  Monitor(gym.make('PortfolioAllocation-v0', **eval_kwargs),'/content/drive/Othercomputers/My Laptop/PortfolioAllocationRL/tensorboard')

In [ ]:
mean_reward, std_reward = evaluate_policy(trained_a2c_model, eval_env, n_eval_episodes=10)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
from scipy import stats as scipy_stats
import pandas as pd
from timeit import default_timer as timer
pfh= trained_a2c_model.env.envs[0].env.portfolio_value_history

returns = pd.DataFrame(pfh)
start = timer()
skew = pd.Series(scipy_stats.skew(returns), index=returns.columns)
end = timer()
print(end - start)
start = timer()
kurtosis = pd.Series(scipy_stats.kurtosis(returns, fisher=False), index=returns.columns)
end = timer()
print(end - start)
print(skew)
print(kurtosis)
returns = pfh
start = timer()
skew = scipy_stats.skew(returns)
end = timer()
print(end - start)
kurtosis = scipy_stats.kurtosis(returns, fisher=False)
end = timer()
print(end - start)
print(skew)
print(kurtosis)

